# The goal of this notebook is to cluster and assign cluster numbers to each word so that they might be useful as features

In [2]:
import os
import json
import random
import pickle

import numpy as np
import pandas as pd

In [3]:
import gensim
from gensim.models.keyedvectors import KeyedVectors

gensim.__version__

'4.3.0'

In [4]:
import sklearn
from sklearn.cluster import KMeans, MiniBatchKMeans
print(sklearn.__version__)

1.3.0


In [5]:
import basic
from basic.nlp.tokenizers import clinical_tokenizers
from basic.nlp.annotation.annotation import Annotation, AnnotatedDocument
from basic.nlp.sequenceutils import get_sentence_bio_tagged_tokens

from madetokenizer import build_made_tokenizer
from madeutils import read_made_data, get_all_sentence_tokens_and_tags, gather_validation_metrics

print('Imported custom BASIC modules')

Imported custom BASIC modules


In [14]:
EMBEDDINGS_BASE_DIR = r'c:\embeddings'

CLUSTERS_BASE_DIR = r'resources/clusters'

PRETRAINED_EMBEDDINGS_FILENAME = r'wikipedia-pubmed-and-PMC-w2v.bin'
#PRETRAINED_EMBEDDINGS_FILENAME = r'pubmed+wiki+pitts-nopunct-lower-cbow-n10.bin'

K_CLUSTERS = 10000
ENABLED_BATCH_KMEANS = True
KMEANS_BATCH_SIZE = 500000

# Load our embeddings

In [15]:
# let's load some pretrained embeddings as well

# NOTE : These embeddings are made available here:
# http://evexdb.org/pmresources/vec-space-models/
#Aashri Note: I hard coded the directory path to this embeddings file instead of using the above global variables as seen in the original code
pretrained_word_vectors = KeyedVectors.load_word2vec_format(r"C:\Users\aaa4026\Desktop\MADE-CRF-master\embeddings\wikipedia-pubmed-and-PMC-w2v.bin", binary=True)  # C binary format
                                                 
print(pretrained_word_vectors)

KeyedVectors<vector_size=200, 5443656 keys>


In [16]:
print(pretrained_word_vectors['the'].shape)

pretrained_embeddings_dimensions = pretrained_word_vectors['the'].shape[0]
print(pretrained_embeddings_dimensions)

(200,)
200


In [17]:
embeddings = pretrained_word_vectors
embeddings_dimensions = pretrained_embeddings_dimensions

In [ ]:
%%time

#changed from .syn0 to .vectors due to .syn0 being depricated in the recent version of the package
word_vectors = pretrained_word_vectors.vectors

# Initalize a k-means object and use it to extract centroids
print('Running K means')

if ENABLED_BATCH_KMEANS:
    print('Using batch KMeans')
    kmeans = MiniBatchKMeans(n_clusters = K_CLUSTERS, 
                         #n_jobs = -2, 
                         batch_size = KMEANS_BATCH_SIZE)
else:
    print('Using original recipe KMeans')
    kmeans = KMeans( n_clusters = K_CLUSTERS, n_jobs = -2 )

cluster_idx = kmeans.fit_predict( word_vectors )

print('K means trained')

Running K means
Using batch KMeans


C:\Users\aaa4026\AppData\Local\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


In [ ]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number
#Aashri edit: took out .wv
word_cluster_map = dict(zip(pretrained_word_vectors.index_to_key, cluster_idx ))

print(list(word_cluster_map.items())[:50])

In [ ]:
print(len(word_cluster_map))

In [ ]:
typename = 'KMeans'
if ENABLED_BATCH_KMEANS:
    typename = 'BatchKmeans'

map_pickle_file_name = '{3}/WordClusters_K{0}_{1}_{2}.pickle'.format(K_CLUSTERS, 
                                                                     typename, 
                                                                     PRETRAINED_EMBEDDINGS_FILENAME.split('.')[0],
                                                                    CLUSTERS_BASE_DIR)

print('Writing cluster map pickle to : {}'.format(map_pickle_file_name))
      
with open(map_pickle_file_name, 'wb') as handle:
    pickle.dump(word_cluster_map, handle, protocol=pickle.HIGHEST_PROTOCOL)
      
print('DONE writing cluster map pickle')